In [2]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [3]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [4]:
df = pd.read_excel('gb_sem_8_hm.xlsx')
df.head()

c:\Users\79898\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [5]:
df.shape

(10000, 3)

In [6]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [7]:
df.USER_ID.nunique()

6324

In [8]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})

In [9]:
df.shape

(7865, 3)

In [10]:
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [11]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [12]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [13]:
df.describe()

,USER_ID,REVENUE
count,7865.000000,7865.000000
mean,4977.856707,0.126442
std,2890.696828,2.613860
min,2.000000,0.000000
25%,2463.000000,0.000000
50%,4965.000000,0.000000
75%,7504.000000,0.000000
max,10000.000000,196.010000


In [17]:
df_new.shape

(4783, 3)

In [23]:
control = df_new[df_new['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = df_new[df_new['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [24]:
control.shape

(2390, 3)

In [28]:
import plotly.express as px
fig = px.histogram(df_new,
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='avg_site_visits_distribution',
                   marginal = 'box',
                   nbins = 10,
                   barmode='overlay')

fig.show()

In [33]:
continious_result(control, treatment, column='REVENUE')

100%|██████████| 10000/10000 [00:07<00:00, 1404.59it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.040483,0.16165,0.712143,"[-0.002, 0.323]",0.122325
